<h1 div class='alert alert-success'><center> TPS-Set: Tunning Hyperparameters (XGB, Catb and LGBM )
 </center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  0. IMPORTAÇÕES </div>

## 0.1. Bibliotecas

In [ ]:
import time
import warnings
import os
import re
import gc
import random
import glob
import optuna
import sklearn.exceptions
import plotly

In [ ]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt 
import seaborn              as sns
import joblib               as jb

In [ ]:
#!conda install catboost

In [ ]:
import torch.nn             as nn

In [ ]:
import lightgbm             as lgb
import catboost             as ctb
import xgboost              as xgb

In [ ]:
from sklearn.model_selection       import train_test_split,StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing         import QuantileTransformer
from sklearn                       import metrics
from sklearn.ensemble              import RandomForestClassifier

In [ ]:
from optuna.samplers               import TPESampler
from optuna.visualization          import plot_edf
from optuna.visualization          import plot_optimization_history
from optuna.visualization          import plot_parallel_coordinate
from optuna.visualization          import plot_param_importances
from optuna.visualization          import plot_slice
from optuna.visualization          import plot_intermediate_values
from optuna.visualization          import plot_contour

In [ ]:
!mkdir Data
!mkdir Data/pkl
!mkdir Data/sumbmission
!mkdir model
!mkdir model/preds
!mkdir model/optuna

## 0.2. Funções

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
     
    pd.options.display.max_columns = None
    
    optuna.logging.set_verbosity(optuna.logging.WARNING)
      
    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

    #pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.rcParams['font.size'] = 12
    plt.title('Precision Recall vs threshold')
    plt.xlabel('Threshold')
    plt.legend(loc="lower left")
    
    plt.grid(True)


In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls[:-1], precisions[:-1], "b-", label="Precision")
    
    plt.rcParams['font.size'] = 12
    plt.title('Precision vs recall')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:

def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('XGBR ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def get_df_cv(path):

    name_file_pkl     = glob.glob(path + '*.pkl.z')
    dic_preds_mdl_pkl = dict()

    for p_name in name_file_pkl:    
        y_model_pkl_name_col  = p_name.replace(path, '').replace('.pkl.z','') 
        y_model_pkl           = jb.load(p_name)   

        dic_preds_mdl_pkl[y_model_pkl_name_col] = y_model_pkl

    return pd.DataFrame(dic_preds_mdl_pkl)

In [ ]:
def graf_corr(df):
    
    df = df.corr().round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(16,16))
    ax = sns.heatmap(df, annot=True, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");


In [ ]:
def correlation(dataset, threshold):

    col_corr    = set()  # Conjunto de todos os nomes de colunas correlacionadas
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # estamos interessados no valor coeficiente absoluto
                colname = corr_matrix.columns[i]        # obtendo o nome da coluna
                col_corr.add(colname)
    
    return col_corr

## 0.3. Carregar Dados

In [ ]:
path = '/content/drive/MyDrive/kaggle/07. Tabular Playground Series - Ago 2021/'
path = '../input/tpsset21-001/'

In [ ]:
df3_train     = jb.load(path + 'df3_train.pkl.z')
df3_test      = jb.load(path + 'df3_test.pkl.z')
df_submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

df3_train.shape, df3_test.shape

In [ ]:
df3_train.head()

In [ ]:
df3_train.info() , df3_test.info()  

### 0.3.1. Reduzindo memória
taken from: https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro

In [ ]:
df3_train = reduce_memory_usage(df3_train)
df3_test  = reduce_memory_usage(df3_test)

df3_train.shape, df3_test.shape

In [ ]:
df3_train.info(), df3_test.info()

# <div class="alert alert-success"> 1.  TUNNING </div>

In [ ]:
X      = df3_train.drop(['claim'], axis=1)    
y      = df3_train["claim"].copy()
X_test = df3_test

X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                      y, 
                                                      test_size    = 0.2, 
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 12359)



print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X.shape, y.shape, X_test.shape) 

In [ ]:
scaler       = QuantileTransformer(output_distribution='normal' , random_state=0)
X_test_sc_qt = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

X_test_sc_qt.shape

## 1.1. Classe Tunning 

In [ ]:
path = ''

class TunningModels(nn.Module):

    def __init__(self, name_model, X_trn_, y_trn_, X_ts_, feature_=None, seed_=12359):
        
        super(TunningModels,self).__init__() 
        
        self.name_clf  = name_model
        self.X_trn     = X_trn_
        self.y_trn     = y_trn_
        self.X_ts      = X_ts_         
        self.feature   = feature_
        self.seed      = seed_
        
    def recover_prediction_first_level():
        
        preds_train1 = glob.glob("model/train/*.pkl.z")
        preds_test   = glob.glob("model/test/*.pkl.z")
        preds_val1   = glob.glob("model/valid/*.pkl.z")

        df_train1     = []
        scores_traint = dict()

        for p_name in preds_train1:    
            p    = jb.load(p_name)
            p_df = pd.DataFrame(p, columns=[p_name.replace('model/train\\', '')])    
            df_train1.append(p_df)    
            scores_traint[p_name] = f1_score(y_train1, (p_df>.5))

        df_val1     = [] 
        scores_val1 = dict()
        for p_name in preds_val1:    
            p    = jb.load(p_name)
            p_df = pd.DataFrame(p, columns=[p_name.replace('model/valid\\', '')])    
            df_val1.append(p_df)    
            scores_val1[p_name] = f1_score(y_val1, (p_df>.5))

        df_test     = [] 
        scores_test = dict()
        for p_name in preds_test:    
            p         = jb.load(p_name)
            p_df_test = pd.DataFrame(p, columns=[p_name.replace('model/test\\', '')])    
            df_test.append(p_df_test)

        df_train1 = pd.concat(df_train1, axis=1)
        df_val1   = pd.concat(df_val1, axis=1)
        df_test   = pd.concat(df_test, axis=1)

        return df_train1, df_val1, df_test.shape
        
    def delete_files(namefile):

        path = ['model/train', 'model/test', 'model/valid', 'model/params', 'model/score',
                'model/test_f', 'model/cv_model', 'model/preds', 'model/optuna']

        for path_ in path:
            for raiz, diretorios, arquivos in os.walk(path_):
                for arquivo in arquivos:
                    if arquivo.startswith(namefile):
                        os.remove(os.path.join(raiz, arquivo))
 
    def logging_callback(study, frozen_trail):
        prev_best = study.user_attrs.get('prev_best', None)
        if prev_best != study.best_value:
            study.set_user_attr('prev_best', study.best_value)
            print(f"Trail {frozen_trail.number} finished with best value {frozen_trail.value}")

    def cross_valid(model, model_name_, X_, y_, X_test_, type_model=1, 
                    feature=None, seed=None, tunning=1):
    
        n_splits    = 5
        n_repeats   = 1
        y_hat_test  = None
        auc         = []
        lloss       = []
        f1          = []

        if feature!=None:         
            X_      = X_[feature]
            X_test_ = X_test_[feature]
        
        kf = RepeatedStratifiedKFold(n_splits     = n_splits, 
                                     n_repeats    = n_repeats,                                   
                                     random_state = seed)
        
        # https://stackoverflow.com/questions/65318931/stratifiedkfold-vs-kfold-in-scikit-learn
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
            

        clf_name = model.__class__.__name__

        print('='*70)
        print('Training model: {}'.format(clf_name))
        print('='*70)
        
        scaler = QuantileTransformer(output_distribution='normal', random_state=seed)
        
        cols_model_base = ['fe_f50_zero', 'fe_f5_zero', 'fe_n_missing', 'cluster', 
                           'fe_std', 'f105', 'f102', 'f22', 'f79', 'f106', 'f71', 
                           'f77', 'f69']

        for fold, (train_idx, valid_idx) in enumerate(kf.split(X_, y_, groups=y_)): 

            # Obtenha os dados de treino e 
            # validação através de indice
            X_trn, y_trn = X_.iloc[train_idx], y_.iloc[train_idx]
            X_val, y_val = X_.iloc[valid_idx], y_.iloc[valid_idx]

            # Scaler 
            X_trn  = pd.DataFrame(scaler.fit_transform(X_trn), columns=X_trn.columns)
            X_val  = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)

            # Treinar o modelo baseline
                        
            #params_md_baseline = {'random_state': 0,          
            #                      'predictor'   : 'gpu_predictor',
            #                      'tree_method' : 'gpu_hist',
            #                      'eval_metric' : 'auc'}
            #
            #model_base = xgb.XGBClassifier(**params_md_baseline)
            #model_base.fit(X_trn[cols_model_base], y_trn)
#
            #threshold = .5
#
            #y_hat_prob_tr_mb = (model_base.predict_proba(X_trn[cols_model_base])[:, 1]>threshold).astype(int) 
            #y_hat_prob_vl_mb = (model_base.predict_proba(X_val[cols_model_base])[:, 1]>threshold).astype(int) 
            #y_hat_prob_ts_mb = (model_base.predict_proba(X_test_[cols_model_base])[:, 1]>threshold).astype(int) 
#
            #X_trn['fe_md_baseline']   = y_hat_prob_tr_mb 
            #X_val['fe_md_baseline']   = y_hat_prob_vl_mb
            #X_test_['fe_md_baseline'] = y_hat_prob_ts_mb

            # Crie e ajuste um novo modelo 
            # usando os melhores parâmetros.        
            if type_model==1:
                model.fit(X_trn, y_trn,
                          eval_set              = [(X_val, y_val)],                          
                          early_stopping_rounds = 100 ,
                          verbose               = False)
                
            if type_model==2:
                model.fit(X_trn, y_trn)

            # Predição na validação
            y_hat_val_prob = model.predict_proba(X_val)[:, 1]
            y_hat_val      = (y_hat_val_prob >.5).astype(int) 

            # Métricas 
            auc_      = metrics.roc_auc_score(y_val, y_hat_val_prob)                    
            f1_score_ = metrics.f1_score(y_val, y_hat_val)        
            log_loss_ = metrics.log_loss(y_val, y_hat_val_prob)
            
            # Salvar as métricas
            auc.append(auc_)
            f1.append(f1_score_)
            lloss.append(log_loss_)

            # Use o modelo treinado para 
            # 1 / n_splits das previsões de saída.
            if y_hat_test is None:        
                y_hat_test  = model.predict_proba(X_test_)[:, 1]        
            else:        
                y_hat_test  += model.predict_proba(X_test_)[:, 1]

            msg = 'Fold {}/{} AUC: {:2.5f} - F1: {:2.5f} - L. loss: {:2.5f}'
            print(msg.format(fold+1, n_splits*n_repeats, auc_, f1_score_, log_loss_))

            gc.collect()

        y_hat_test /= (n_splits*n_repeats)
        auc_mean    = np.mean(auc)
        auc_std     = np.std(auc)
        lloss_mean  = np.mean(lloss)
        f1_mean     = np.mean(f1)
        msg         = '[Mean Fold] AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. Loss: {:.5f}'
        
        # 
        if tunning==1:
            path_name = path + 'model/preds/' + model_name_.format(auc_mean, seed)                        
            jb.dump(y_hat_test, path_name)
            score = auc_mean
        else: 
            score = y_hat_test
            
        gc.collect()

        print('-'*70)
        print(msg.format(auc_mean, auc_std, f1_mean, lloss_mean))
        print('='*70)
        print('')
                    
        return score 
        
    def xgb(self, trial):
                
        params = {'n_estimators'      : trial.suggest_int('n_estimators', 1000, 10000),
                  'max_depth'         : trial.suggest_int('max_depth', 3, 10),            
                  'min_child_weight'  : trial.suggest_int('min_child_weight', 10, 250),
                  'subsample'         : trial.suggest_uniform('subsample', .65, .90),
                  'colsample_bynode'  : trial.suggest_uniform('colsample_bynode', .65, .8),                              
                  'learning_rate'     : trial.suggest_uniform('learning_rate', 0.01, 0.3),
                  'colsample_bytree'  : trial.suggest_uniform('colsample_bytree', 0.3, 0.85),
                  'reg_lambda'        : trial.suggest_int('reg_lambda', 2, 100),             
                  'reg_alpha'         : trial.suggest_int('reg_alpha', 1, 50),             
                  'eta'               : trial.suggest_uniform('eta', 0.01, 0.13),
                  'alpha'             : trial.suggest_uniform('alpha', .3, .7), 
                  'gamma'             : trial.suggest_uniform('alpha', 10.7, 15.5), 
                 }
                
                        
        mdl = xgb.XGBClassifier(**params, 
                                objective         = 'binary:logistic',                  
                                predictor         = 'gpu_predictor',
                                tree_method       = 'gpu_hist',
                                eval_metric       = 'auc', 
                                random_state      = self.seed  
                               )
        
        random     = str(np.random.rand(1)[0]).replace('.','')
        name_model = self.name_clf + '_auc_{:2.5f}_{}_' + random + '.pkl.z'  
        
        score = TunningModels.cross_valid(model         = mdl, 
                                          model_name_   = name_model,
                                          X_            = self.X_trn, 
                                          y_            = self.y_trn,  
                                          X_test_       = self.X_ts,
                                          type_model    = 2, 
                                          feature       = self.feature,
                                          seed          = self.seed)
             
        return score
   
    def lgb(self, trial):
        
        params = {'learning_rate'     : trial.suggest_float('learning_rate', 1e-5, 0.1),               
                  'max_depth'         : trial.suggest_int('max_depth', 3, 12),
                  'n_estimators'      : trial.suggest_int('n_estimators', 500, 10000),
                  'num_leaves'        : trial.suggest_int('num_leaves', 8, 356),
                  'min_child_samples' : trial.suggest_int('min_child_samples', 2, 3000),
                  'feature_fraction'  : trial.suggest_uniform('feature_fraction', 0.75, 0.95),
                  'bagging_fraction'  : trial.suggest_uniform('bagging_fraction', 0.75, 0.95),
                  'bagging_freq'      : trial.suggest_int('bagging_freq', 3, 7),
                  'reg_alpha'         : trial.suggest_int('reg_alpha', .1, .90),
                  'reg_lambda'        : trial.suggest_int('reg_lambda', 1e-5, 1e-2),                   
                  'lambda_l1'         : trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                  'lambda_l2'         : trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),           
                }
        
        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
        
        mdl = lgb.LGBMClassifier(**params, 
                                 objective     = 'binary',                   
                                 metric        = 'auc',
                                 verbosity     = -1,
                                 boosting_type = 'gbdt',
                                 class_weight  = 'balanced',
                                 #callbacks     = [pruning_callback],
                                 random_state  = self.seed, 
                                 n_jobs        = -1,
                                 device       = 'gpu',                                 
                                )
        
        random     = str(np.random.rand(1)[0]).replace('.','')
        name_model = self.name_clf + '_auc_{:2.5f}_{}_' + random + '.pkl.z'  
        
        score = TunningModels.cross_valid(model         = mdl, 
                                          model_name_   = name_model,
                                          X_            = self.X_trn, 
                                          y_            = self.y_trn,  
                                          X_test_       = self.X_ts,
                                          type_model    = 1, 
                                          feature       = self.feature,
                                          seed          = self.seed)
        
        return score
    
    def ctb(self, trial):
                
        params = {
                  'n_estimators'              : trial.suggest_int('n_estimators', 1000, 7000),
                  'max_depth'                  : trial.suggest_int('max_depth', 2, 12),
                  'iterations'                : trial.suggest_int('iterations', 1000, 7000),
                  'od_wait'                   : trial.suggest_int('od_wait', 500, 2000),
                  'learning_rate'             : trial.suggest_uniform('learning_rate', 1e-5, 0.1),
                  'reg_lambda'                : trial.suggest_uniform('reg_lambda',1e-5, 1e-2),
                  'subsample'                 : trial.suggest_uniform('subsample', 0, 1),
                  'random_strength'           : trial.suggest_uniform('random_strength', 10, 50),
                  'depth'                     : trial.suggest_int('depth', 3, 12),
                  'min_data_in_leaf'          : trial.suggest_int('min_data_in_leaf', 1, 30),
                  'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 15),
                }
        
        mdl = ctb.CatBoostClassifier(**params, 
                                     eval_metric  = 'AUC',
                                     task_type    = 'GPU',
                                    )
              
        random     = str(np.random.rand(1)[0]).replace('.','')
        name_model = self.name_clf + '_auc_{:2.5f}_{}_' + random + '.pkl.z'  
        
        score = TunningModels.cross_valid(model         = mdl, 
                                          model_name_   = name_model,
                                          X_            = self.X_trn, 
                                          y_            = self.y_trn,  
                                          X_test_       = self.X_ts,
                                          type_model    = 1, 
                                          feature       = self.feature,
                                          seed          = self.seed)
        
        return score
    
    
    def rf(self, trial):
                
        params = {'max_depth'           : trial.suggest_int('max_depth', 4, 12),
                  'n_estimators'        : trial.suggest_int('n_estimators', 10, 1000),
                  'max_samples'         : trial.suggest_int('max_samples', 50 , 1200), 
                  #'min_sample_split'    : trial.suggest_int('min_sample_split', 100 , 1000),
                  #'max_terminal_nodes'  : trial.suggest_float('max_terminal_nodes', .01 , .8),                  
                  #'min_samples_lea'     : trial.suggest_int('min_samples_lea', 100 , 1000),
                  'max_features'        : trial.suggest_float('max_features', .65 , .9), 
                }
     
            
        mdl = rf_model = RandomForestClassifier(**params, n_jobs=-1)
                      
        random     = str(np.random.rand(1)[0]).replace('.','')
        name_model = self.name_clf + '_auc_{:2.5f}_{}_' + random + '.pkl.z'  
        
        score = TunningModels.cross_valid(model         = mdl, 
                                          model_name_   = name_model,
                                          X_            = self.X_trn, 
                                          y_            = self.y_trn,  
                                          X_test_       = self.X_ts,
                                          type_model    = 2, 
                                          feature       = self.feature,
                                          seed          = self.seed)
        
        return score
    
     
    def knn(self, trial):
                
        params = {'n_neighbors' : trial.suggest_int('n_neighbors', 4, 500),
                  #'metric'      : trial.suggest_categorical ('max_samples', ['euclidean', 'minkowski','seuclidean'])
                }
        
        mdl = KNeighborsClassifier(**params)
        
        random     = str(np.random.rand(1)[0]).replace('.','')
        name_model = self.name_clf + '_auc_{:2.5f}_{}_' + random + '.pkl.z'  
        
        feature= ['fe_n_missing', 'fe_f5_zero', 'fe_f50_zero', 'fe_mean',
                  'fe_median', 'fe_std', 'fe_skew', 'cluster']
        
        score = TunningModels.cross_valid(model         = mdl, 
                                          model_name_   = name_model,
                                          X_            = self.X_trn, 
                                          y_            = self.y_trn,  
                                          X_test_       = self.X_ts,
                                          type_model    = 2, 
                                          feature       = feature,
                                          seed          = self.seed)
        
        return score

Vamos aplicar o scaler nos dados de teste. 

## 1.2. XGB

### 1.2.1. Tunning 

In [ ]:
# %%time
# 
# SEED = 12359 
# 
# name_model_xgb = 'xgb_001_tun'
# TunningModels.delete_files(name_model_xgb)
#     
# # Inicialize a classe do modelo de otimização
# modelOpt = TunningModels(name_model = name_model_xgb, 
#                          X_trn_     = X, 
#                          y_trn_     = y, 
#                          X_ts_      = X_test_sc_qt,                                     
#                          feature_   = None)
# 
# study_xgb = optuna.create_study(direction = 'maximize',
#                                 sampler   = optuna.samplers.TPESampler(seed=SEED),
#                                 pruner    = optuna.pruners.MedianPruner(n_warmup_steps=10),
#                                 study_name= 'xgb_tuning'
#                                ) 
# 
# study_xgb.optimize(modelOpt.xgb, 
#                    n_trials  = 10, 
#                   )
# 
# auc_best_xgb = study_xgb.best_value 
# params_xgb   = study_xgb.best_params 
# path_name    = path + 'model/optuna/' + name_model_xgb + '_{:2.5f}.pkl.z'.format(auc_best_xgb) 
# 
# print('')
# print('Best average accuracy: {:2.5f}'.format(auc_best_xgb))
# print('Best parameters: {}'.format(params_xgb))
# 
# jb.dump(study_xgb, path_name)

Agora vamos recuperar os melhores parametros do tunning para treinar um modelo.

In [ ]:
#params_xgb

### 1.2.2. Analise de hiperparâmetros

- Visualize the optimization history

In [ ]:
#plot_optimization_history(study_xgb)

- Visualize the learning curves of the trials

In [ ]:
#plot_intermediate_values(study_xgb)

- isualize relacionamentos de parâmetros de alta dimensão. 

In [ ]:
#plot_parallel_coordinate(study_xgb)

- Selecione os parâmetros para visualizar.

In [ ]:
#plot_parallel_coordinate(study_xgb, params=["max_depth", "n_estimators", 'reg_lambda'])

- Visualize relacionamentos de hiperparâmetros.

In [ ]:
#plot_contour(study_xgb)

- Selecione os parâmetros para visualizar.

In [ ]:
#plot_contour(study_xgb, params=["max_depth", "n_estimators", 'reg_lambda'])

- Visualize hiperparâmetros individuais como gráfico de fatias.

In [ ]:
#plot_slice(study_xgb)

- Selecione os parâmetros para visualizar.

In [ ]:
#plot_slice(study_xgb, params=["max_depth", "n_estimators", 'reg_lambda'])

- Visualize as importâncias dos parâmetros.  

In [ ]:
#plot_param_importances(study_xgb)

- Quais hiperparâmetros estão afetando a duração do ensaio com a importância dos hiperparâmetros.

In [ ]:
#optuna.visualization.plot_param_importances(study_xgb, 
#                                            target=lambda t: t.duration.total_seconds(), 
#                                            target_name="duration")

- Visualize a função de distribuição empírica

In [ ]:
#plot_edf(study_xgb)

### 1.1.3. Modelo Final

In [ ]:
#%%time
#model_xgb_001 = xgb.XGBClassifier(**params_xgb, 
#                                  objective         = 'binary:logistic',                  
#                                  predictor         = 'gpu_predictor',
#                                  tree_method       = 'gpu_hist',
#                                  eval_metric       = 'auc', 
#                                  random_state      = SEED
#                                 )
#
#y_hat_test_xgb_001 = TunningModels.cross_valid(model       = model_xgb_001, 
#                                               model_name_ = name_model_xgb, 
#                                               X_          = X_train, 
#                                               y_          = y_train, 
#                                               X_test_     = X_test_sc_qt, 
#                                               type_model  = 1, 
#                                               feature     = None,
#                                               seed        = SEED, 
#                                               tunning     = 0
#                                               )
#
#model_xgb = model_xgb_001.fit(X_train, y_train)
#y_hat_xgb = model_xgb.predict(X_valid)
#
#print(metrics.classification_report(y_valid, y_hat_xgb))

In [ ]:
#metrics.plot_confusion_matrix(model_xgb, 
#                              X_valid, 
#                              y_valid,
#                              cmap   = 'inferno')
#plt.title('Confusion matrix')
#plt.grid(False)
#plt.show()

Precision Recall versus the decision threshold

In [ ]:
#y_pred = y_hat_xgb
#y_true = y_valid
#precisions, recalls, thresholds = metrics.precision_recall_curve(y_true, y_pred)
#
#plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
#plt.show()

In [ ]:
#plot_precision_vs_recall(precisions, recalls)
#plt.show()

ROC curve


In [ ]:
#y_true=y_valid
#y_pred=y_hat_xgb
#
#fpr, tpr, thresholds = metrics. roc_curve(y_true, y_pred)
#
#plot_roc_curve(fpr, tpr, label="XGB")
#plt.show()


### 1.1.4. Gerar Submission

In [ ]:
#submission = pd.DataFrame({'id': df_submission.id, 'loss': y_hat_test_xgb_001})
#submission.to_csv(path + 'Data/sumbmission/001_xgx_submission_tunning.csv', index=False)

## 1.3. LGBM

### 1.3.1. Tunning 

In [ ]:
# %%time
# SEED = 12359 
# 
# name_model_lgb = 'lgb_001_tun'
# TunningModels.delete_files(name_model_lgb)
#     
# # Inicialize a classe do modelo de otimização
# modelOpt = TunningModels(name_model = name_model_lgb, 
#                          X_trn_     = X, 
#                          y_trn_     = y, 
#                          X_ts_      = X_test_sc_qt,                                     
#                          feature_   = None)
# 
# study_lgb = optuna.create_study(direction = 'maximize',
#                                 sampler   = optuna.samplers.TPESampler(seed=SEED),
#                                 pruner    = optuna.pruners.MedianPruner(n_warmup_steps=10),
#                                 study_name= 'lgbm_tuning'
#                                ) 
# 
# study_lgb.optimize(modelOpt.lgb,n_trials=10)
# 
# auc_best_lgb = study_lgb.best_value 
# params_lgb   = study_lgb.best_params 
# path_name    = path + 'model/optuna/' + name_model_lgb + '_{:2.5f}.pkl.z'.format(auc_best_lgb) 
# 
# print('')
# print('Best average accuracy: {:2.5f}'.format(auc_best_lgb))
# print('Best parameters: {}'.format(params_lgb))
# 
#jb.dump(study_lgb, path_name)

### 1.3.2. Analise de hiperparâmetros
- referência: https://optuna.readthedocs.io/en/latest/tutorial/10_key_features/005_visualization.html

**- Visualize o histórico de otimização.**

In [ ]:
#plot_optimization_history(study_lgb)

**- Visualize relacionamentos de parâmetros de alta dimensão.**

In [ ]:
#plot_parallel_coordinate(study_lgb)

In [ ]:
#plot_parallel_coordinate(study_lgb, params=['max_depth','learning_rate', 'num_leaves'])

**- Visualize hiperparâmetros individuais como gráfico de fatias.**

In [ ]:
#plot_slice(study_lgb)

In [ ]:
#plot_slice(study_lgb)

**- Selecione os parâmetros para visualizar.**

In [ ]:
#plot_slice(study_lgb, params=['max_depth','learning_rate', 'num_leaves'])

**- Visualize as importâncias dos parâmetros.**

In [ ]:
#optuna.visualization.plot_param_importances(study_lgb, 
#                                            target=lambda t: t.duration.total_seconds(), 
#                                            target_name="duration"
#                                           )

**- Saiba quais hiperparâmetros estão afetando a duração do ensaio com a importância dos hiperparâmetros.**

In [ ]:
#plot_param_importances(study_lgb, 
#                       target      = lambda t: t.duration.total_seconds(),
#                       target_name = "duration")

**- Visualize a função de distribuição empírica.**

In [ ]:
#plot_edf(study_lgb)

### 1.3.3. Modelo Final

In [ ]:
#%%time
#model_lgb_001      = lgb.LGBMClassifier(**params_lgb,
#                                        objective     = 'binary',                   
#                                        metric        = 'auc',
#                                        verbosity     = -1,
#                                        boosting_type = 'gbdt',
#                                        class_weight  = 'balanced',                                        
#                                        random_state  = SEED, 
#                                        n_jobs        = -1,
#                                        device       = 'gpu',                                 
#                                        )
#                                        
#y_hat_test_lgb_001 = TunningModels.cross_valid(model       = model_lgb_001, 
#                                               model_name_ = name_model_lgb, 
#                                               X_          = X_train, 
#                                               y_          = y_train, 
#                                               X_test_     = X_test_sc_qt, 
#                                               type_model  = 1, 
#                                               feature     = None,
#                                               seed        = SEED, 
#                                               tunning     = 0
#                                               )
#
#model_lgb = model_lgb_001.fit(X_train, y_train)
#y_hat_lgb = model_lgb.predict(X_valid)

#print(metrics.classification_report(y_valid, y_hat_lgb))

### 1.3.2. Analise

In [ ]:
#metrics.plot_confusion_matrix(model_lgb, 
#                              X_valid, 
#                              y_valid,
#                              cmap   = 'inferno')
#plt.title('Confusion matrix')
#plt.grid(False)
#plt.show()

### 1.3.3. Gerar Submission

In [ ]:
#submission = pd.DataFrame({'id': df_submission.id, 'loss': y_hat_test_lgb_001})
#submission.to_csv(path + 'Data/sumbmission/001_lgb_submission_tunning.csv', index=False)

## 1.4. RF

### 1.4.1. Tunning 

In [ ]:
#%%time
#SEED = 12359 
#
#name_model_rf = 'rf_001_tun'
#TunningModels.delete_files(name_model_rf)
#    
#modelOpt = TunningModels(name_model = name_model_rf, 
#                         X_trn_     = X, 
#                         y_trn_     = y, 
#                         X_ts_      = X_test_sc_qt,                                     
#                         feature_   = None)
#
#study_rf = optuna.create_study(direction = 'maximize',
#                            sampler   = optuna.samplers.TPESampler(seed=SEED),
#                            pruner    = optuna.pruners.MedianPruner(n_warmup_steps=10),
#                            study_name= 'rf_tuning'
#                           ) 
#
#study_rf.optimize(modelOpt.rf, n_trials=10)
#
#auc_best_rf = study_rf.best_value 
#params_rf   = study_rf.best_params 
#path_name    = path + 'model/optuna/' + name_model_rf + '_{:2.5f}.pkl.z'.format(auc_best_rf) 
#
#print('')
#print('Best average accuracy: {:2.5f}'.format(auc_best_rf))
#print('Best parameters: {}'.format(params_rf))
#print('')

#jb.dump(study_rf, path_name)

### 1.4.2. Modelo Final

In [ ]:
#model_rf_001      = RandomForestClassifier(**params_rf)
#y_hat_test_rf_001 = TunningModels.cross_valid(model       = model_rf_001, 
#                                               model_name_ = name_model_rf, 
#                                               X_          = X_train, 
#                                               y_          = y_train, 
#                                               X_test_     = X_test_sc_qt, 
#                                               type_model  = 2, 
#                                               feature     = None,
#                                               seed        = SEED, 
#                                               tunning     = 0
#                                               )
#
#model_rf = model_rf_001.fit(X_train, y_train)
#y_hat_rf = model_rf.predict(X_valid)
#
#print(metrics.classification_report(y_valid, y_hat_rf))

### 1.4.2. Analise

In [ ]:
#metrics.plot_confusion_matrix(model_rf, 
#                              X_valid, 
#                              y_valid,
#                              cmap   = 'inferno')
#plt.title('Confusion matrix')
#plt.grid(False)
#plt.show()

### 1.4.3. Gerar Submission

In [ ]:
#submission = pd.DataFrame({'id': df_submission.id, 'loss': y_hat_test_rf_001})
#submission.to_csv(path + 'Data/sumbmission/001_rf_submission_tunning.csv', index=False)

https://www.kaggle.com/vishwas21/tps-sep-21-tuning-xgb-catb-lgbm-and-stacking#Tuning-Light-GBM-Classifier-using-Optuna

# 15. Ensable 

Para ensable vamos utilizar as previsões geradas nos processos anteriores, que foram comentados, pois na versão anterior no notebook não foi processado até o final, como quero saber o resultado e ter uma ideia como estou estou na competião vamos gerar as submissões. 

## 15.1. Recuparar as submissions  

In [ ]:
#df_sub_lgb = pd.read_csv('../input/tpsset21003/001_lgb_submission_tunning.csv')
#df_sub_xgb  = pd.read_csv('../input/tpsset21003/001_xgx_submission_tunning.csv')
#
#df_sub_lgb.shape, df_sub_xgb.shape

### 15.1.2. Predição com a média ponderada

In [ ]:
#submission = pd.DataFrame({'id': df_submission.id, 'claim': df_sub_lgb.claim})
#submission.to_csv(path + 'Data/sumbmission/001_lgb_submission_01.csv', index=False)

In [ ]:
#y_pred_ensable_01 = df_sub_lgb.claim * 0.8 + df_sub_xgb.claim * .2
#y_pred_ensable_01

# score kaggle: 0.81359 com .5
# scare kagle: 0.81631 

### 15.3.1. Gerar Submission

In [ ]:
#submission = pd.DataFrame({'id': df_submission.id, 'claim': y_pred_ensable_01})
#submission.to_csv(path + 'Data/sumbmission/001_ensable_submission_02.csv', index=False)

## 15.2. Recuparar dataset
Aqui vamos recupar todas a previsões realizadas na validação cruzada para gerar uma nova submission.

In [ ]:
df_cv_1 = get_df_cv('../input/tpsset21002/')
df_cv_2 = get_df_cv('../input/psset21003/')
df_cv_1.shape, df_cv_2.shape

In [ ]:

df_cv = pd.concat([df_cv_1, df_cv_2], axis=1)
df_cv.shape

Como podemos observar, temos 22 previsões realizadas na validação cruzada, é a mesma coisa que dizer que temos 22 modelos, vamos dar uma olhada no dataset.

In [ ]:
df_cv.head()

### 15.2.1. Descritiva 

In [ ]:
df_cv.describe()

In [ ]:
df_cv.mean(axis=1)

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'claim': df_cv.mean(axis=1) })
submission.to_csv(path + 'Data/sumbmission/001_ensable_submission_05.csv', index=False) 

### 15.2.3. Correlação

In [ ]:
graf_corr(df_cv)

Temos muitas previsões autocorrelacionadas, vamos fazer a exclusão de algumas. 

In [ ]:
corr_features = correlation(df_cv, 0.999)
len(set(corr_features))

In [ ]:
print(corr_features)

In [ ]:
df_cv.drop(labels=corr_features, axis=1, inplace=True)

graf_corr(df_cv)


In [ ]:
df_cv.columns

### 15.4.1. Gerar Submission

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'claim': df_cv.mean(axis=1)})
submission.to_csv(path + 'Data/sumbmission/001_ensable_submission_06.csv', index=False) 

# submissão no kaggle: 0.81154 

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'claim': df_cv.filter(regex=r'auc_0.81', axis=1).mean(axis=1) })
submission.to_csv(path + 'Data/sumbmission/001_ensable_submission_07.csv', index=False) 

# submissão no kaggle: 0.81154  